## Multiple SceneObjects

Test fixed and tracking sceneObjects multiple on teh same scene.

Try appendtoScene too.




In [1]:
try:
    from bifacial_radiance import RadianceObj, AnalysisObj
except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')
    # Simple example system using Radiance.
import numpy as np

testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Demo2'



In [2]:
# GEnerating Standard FIXED SETUP
demo = RadianceObj("MultipleObj", path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(0.62)
epwfile = demo.getEPW(lat = 37.5, lon = -77.6)    
metdata = demo.readEPW('EPWs\\USA_VA_Richmond.Intl.AP.724010_TMY.epw') 
#metdata = demo.readEPW('EPWs\\USA_VA_Richmond.Intl.AP.724010_TMY.epw') # read in the weather data directly
fullYear = True
#demo.genCumSky(demo.epwfile) # entire year.
demo.gendaylit(metdata,4020)  # Noon, June 17th 
module_type = 'Prism Solar Bi60 landscape' 
demo.makeModule(name=module_type,y=1,x=1.7,bifi = 0.90)
sceneDict = {'tilt':10,'pitch':3,'clearance_height':0.2,'azimuth':180, 'nMods': 10, 'nRows': 2, 'appendRadfile':True} 
sceneObj1 = demo.makeScene(module_type,sceneDict)  


path = C:\Users\sayala\Documents\RadianceScenes\Demo2
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw  ... OK!
('\nModule Name:', 'Prism_Solar_Bi60_landscape')
REWRITING pre-existing module file. 
Module Prism Solar Bi60 landscape successfully created
Radfile APPENDED!


In [3]:
# Checking values after Scene
print sceneObj1.radfiles
print sceneObj1.modulefile
print sceneObj1.gcr
print demo.getfilelist()

objects\Prism_Solar_Bi60_landscape_0.2_3_10x2.rad
objects\Prism_Solar_Bi60_landscape.rad
0.333333333333
['materials\\ground.rad', 'skies\\sky2_37.5_-77.33_06_17_13.rad', 'objects\\Prism_Solar_Bi60_landscape_0.2_3_10x2.rad']


In [4]:
# Creating a different Scene. Same Module, different values.
sceneDict2 = {'tilt':30,'pitch':5,'clearance_height':1,'azimuth':180, 'nMods': 5, 'nRows': 1, 'originx': 0, 'originy': 5, 'appendRadfile':True} 
module_type2='Module2'
demo.makeModule(name=module_type2,x=1,y=1.6, numpanels=2, ygap=0.15)
sceneObj2 = demo.makeScene(module_type2,sceneDict2)  


('\nModule Name:', 'Module2')
REWRITING pre-existing module file. 
Module Module2 successfully created
Radfile APPENDED!


In [5]:
# Checking values for both scenes after creating new SceneObj
print sceneObj1.radfiles
print sceneObj1.modulefile
print sceneObj1.gcr
print sceneObj2.radfiles
print sceneObj2.modulefile
print sceneObj2.gcr

#getfilelist should have info for the rad file created by each scene obect
print demo.getfilelist()

objects\Prism_Solar_Bi60_landscape_0.2_3_10x2.rad
objects\Prism_Solar_Bi60_landscape.rad
0.333333333333
objects\Module2_1.0_5_5x1.rad
objects\Module2.rad
0.67
['materials\\ground.rad', 'skies\\sky2_37.5_-77.33_06_17_13.rad', 'objects\\Prism_Solar_Bi60_landscape_0.2_3_10x2.rad', 'objects\\Module2_1.0_5_5x1.rad']


In [6]:
octfile = demo.makeOct(demo.getfilelist()) 

Created MultipleObj.oct


### At this point you should be able to go into a command window (cmd.exe) and check the geometry. Example:

### rvu -vf views\front.vp -e .01 MultipleObj.oct
And then proceed happily with your analysis:

In [7]:
# sceneDict is saved for each scene. When calling the Analysis, you should refernce the scene object you want.
sceneObj1.sceneDict

{'appendRadfile': True,
 'axis_tilt': 0,
 'azimuth': 180,
 'clearance_height': 0.2,
 'nMods': 10,
 'nRows': 2,
 'originx': 0,
 'originy': 0,
 'pitch': 3,
 'tilt': 10}

In [8]:
sceneObj2.sceneDict

{'appendRadfile': True,
 'axis_tilt': 0,
 'azimuth': 180,
 'clearance_height': 1,
 'nMods': 5,
 'nRows': 1,
 'originx': 0,
 'originy': 5,
 'pitch': 5,
 'tilt': 30}

In [9]:
analysis = AnalysisObj(octfile, demo.basename)  
frontscan, backscan = analysis.moduleAnalysis(sceneObj1)
frontdict, backdict = analysis.analysis(octfile, "FirstObj", frontscan, backscan)  # compare the back vs front irradiance  
print('Annual bifacial ratio: %0.3f ' %( np.mean(analysis.Wm2Back) / np.mean(analysis.Wm2Front)) )

Linescan in process: FirstObj_Front
Linescan in process: FirstObj_Back
Saved: results\irr_FirstObj.csv
Annual bifacial ratio: 0.169 


In [10]:
# Sanity check for first obect. Since we didn't pass any desired module, it should grab the center module of the center row (rounding down)
# for 2 rows, that is row 1, module 5 ~ indexed at 0, a4.0.a0.PVmodule.....""
print frontdict['x']
print ("")
print frontdict['y']
print ("")
print frontdict['mattype']

[4.824167e-17, 3.618125e-17, 2.412083e-17, 1.206042e-17, -1.232595e-32, -1.206042e-17, -2.412083e-17, -3.618125e-17, -4.824167e-17]

[-0.3939231, -0.2954423, -0.1969616, -0.09848078, 0.0, 0.09848078, 0.1969616, 0.2954423, 0.3939231]

[u'a4.0.a0.PVmodule.6457', u'a4.0.a0.PVmodule.6457', u'a4.0.a0.PVmodule.6457', u'a4.0.a0.PVmodule.6457', u'a4.0.a0.PVmodule.6457', u'a4.0.a0.PVmodule.6457', u'a4.0.a0.PVmodule.6457', u'a4.0.a0.PVmodule.6457', u'a4.0.a0.PVmodule.6457']


In [11]:
analysis = AnalysisObj(octfile, demo.basename)  
frontscan, backscan = analysis.moduleAnalysis(sceneObj2)
frontdict2, backdict2 = analysis.analysis(octfile, "SecondOb", frontscan, backscan)  # compare the back vs front irradiance  
print('Annual bifacial ratio: %0.3f ' %( np.mean(analysis.Wm2Back) / np.mean(analysis.Wm2Front)) )

Linescan in process: SecondOb_Front
Linescan in process: SecondOb_Back
Saved: results\irr_SecondOb.csv
Annual bifacial ratio: 0.358 


In [12]:
# Sanity check for first obect. Since we didn't pass any desired module, it should grab the center module of the center row (rounding down)
# for 1 rows, that is row 0, module 3 ~ indexed at 0, a2.0.a0.PVmodule... and a2.0.a1.PVmodule since it is a 2-UP system.
print frontdict2['x']
print ("")
print frontdict2['y']
print ("")
print frontdict2['mattype']

[1.421171e-16, 1.065878e-16, 7.105854e-17, 3.552927e-17, -4.930381e-32, -3.552927e-17, -7.105854e-17, -1.065878e-16, -1.421171e-16]

[3.839526, 4.129644, 4.419763, 4.709881, 5.0, 5.290119, 5.580237, 5.870356, 6.160474]

[u'a2.0.a0.PVmodule.6457', u'a2.0.a0.PVmodule.6457', u'a2.0.a0.PVmodule.6457', u'a2.0.a0.PVmodule.6457', u'groundplane', u'a2.0.a1.PVmodule.6457', u'a2.0.a1.PVmodule.6457', u'a2.0.a1.PVmodule.6457', u'a2.0.a1.PVmodule.6457']


# OPTION B
Another option is to "append" information to the radfiles itself. 
I usually use this to create "markers" for the geometry to orient myself when doing sanity-checks (for example, marke where 0,0 is, or where 5,0 coordinate is). That is what I am doing here, for the image I'm attaching.

In [13]:
# NOTE: offseting translation by 0.1 so the center of the cube is at the desired coordinate.
name='Post1'
text='! genbox black marker1 0.2 0.2 1 | xform -t -0.1 -0.1 0'
customObject = demo.makeCustomObject(name,text)
demo.appendtoScene(sceneObj1.radfiles, customObject, '!xform -rz 0')

name='Post2'
text='! genbox black marker2 0.2 0.2 2.5 | xform -t -0.1 4.9 0'   
customObject = demo.makeCustomObject(name,text)
demo.appendtoScene(sceneObj2.radfiles, customObject, '!xform -rz 0')
octfile = demo.makeOct(demo.getfilelist()) 


('\nCustom Object Name', 'objects\\Post1.rad')
('\nCustom Object Name', 'objects\\Post2.rad')
Created MultipleObj.oct


At this point you should be able to go into a command window (cmd.exe) and check the geometry. Example:

## rvu -vf views\front.vp -e .01 MultipleObj.oct
